In [350]:
import pandas as pd
import numpy as np
import warnings

m_lt = pd.read_excel('gorilla_test_data.xlsx', sheet_name='meter_list')
ft = pd.read_excel('gorilla_test_data.xlsx', sheet_name='forecast_table')
rate = pd.read_excel('gorilla_test_data.xlsx', sheet_name='rate_table')


rate["aq_max_kwh"].loc[np.isnan(rate["aq_max_kwh"])] = np.inf
# def daily_cost_p(date, meter):
#     return rate, forecast

/var/folders/yd/7fn380h1507crytk9ytqnc440000gn/T/ipykernel_68823/1801911778.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate["aq_max_kwh"].loc[np.isnan(rate["aq_max_kwh"])] = np.inf


,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


ft

In [97]:
meters = m_lt['meter_id']
print(meters)
exit_zone = m_lt['exit_zone'][m_lt['meter_id'] == m_lt['meter_id'][1]]
print(exit_zone)
print('{}'.format(rate['date']))

0    14676236
1    34509937
2    50264822
3    88357331
Name: meter_id, dtype: int64
1    SO1
Name: exit_zone, dtype: object


0      2020-04-01
1      2020-04-01
2      2020-04-01
3      2020-04-01
4      2020-04-01
          ...    
1135   2024-10-01
1136   2024-10-01
1137   2024-10-01
1138   2024-10-01
1139   2024-10-01
Name: date, Length: 1140, dtype: datetime64[ns]


0    2020
1    2020
Name: 0, dtype: int64
<class 'numpy.int64'>
84


0      2020-04-01
1      2020-04-01
2      2020-04-01
3      2020-04-01
4      2020-04-01
          ...    
1135   2024-10-01
1136   2024-10-01
1137   2024-10-01
1138   2024-10-01
1139   2024-10-01
Name: date, Length: 1140, dtype: datetime64[ns]

In [321]:
rate

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000.0,0.4537
1136,2024-10-01,WM2,732000,NaN,0.7534
1137,2024-10-01,WM3,0,73200.0,0.7263
1138,2024-10-01,WM3,73200,732000.0,0.6109


/var/folders/yd/7fn380h1507crytk9ytqnc440000gn/T/ipykernel_68823/1575244424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate["aq_max_kwh"].loc[np.isnan(rate["aq_max_kwh"])] = np.inf


In [332]:
rate

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,inf,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000.0,0.4537
1136,2024-10-01,WM2,732000,inf,0.7534
1137,2024-10-01,WM3,0,73200.0,0.7263
1138,2024-10-01,WM3,73200,732000.0,0.6109


In [347]:
def calculate_cost_energy(m_lt, ft):
    """
    Calculate total consumption and the cost of meter in the forcast period

    Parameters:
    m_lt (pd.DataFrame): meter_list in the given dataset
    ft (pd.DataFrame): forecast_table in the given dataset
    Returns:
    pd.DataFrame: table containing Total_consumption and cost of the given meter
    """
    meter_energy = pd.DataFrame({"Meter":[], 
                            "Total_Consumption":[],
                            "Cost":[]})
    
    k = -1
    kwh_sum = []
    cost_sum = []
    
    
    ft['rates'] = 0
    ft_gr = ft.groupby('meter_id')
    # find the upper limit of max. days according to rate_table
    ed = pd.DataFrame({0:['2021-04-01','2021-10-01']})
    ed[0] = pd.to_datetime(ed[0])
    days_lim = (ed[0][1] - ed[0][0]) / np.timedelta64(1, 'D')
    
    
    for name, group in ft_gr:
        k += 1
        i = 0
        exit_zone = m_lt['exit_zone'].iloc[k]
        aq = m_lt['aq_kwh'].iloc[k]

        indices = group.index

        for ft_date in group['date']:

            if rate['date'].iloc[-1]>=group['date'][indices[i]]:


                cp = rate[(rate['exit_zone']==exit_zone)
                          *(group['date'][indices[i]]>= rate['date'])
                          *((group['date'][indices[i]] - rate['date'])/ np.timedelta64(1, 'D') < days_lim)].index.values

                ac_index = [cp[j] for j in range(len(cp)) if rate['aq_min_kwh'][cp[j]] <= aq <= rate['aq_max_kwh'][cp[j]]]
#                 print('\n',ac_index)
#                 print(cp, 'cp \n')
#                 print(group['date'][indices[i]], 'forcast_date\n')
                group['rates'].loc[indices[i]] = rate['rate_p_per_kwh'].iloc[ac_index[-1]]
                i+=1



            else:
                break

        kwh_sum.append(sum(group['kwh']))
        cost_sum.append(sum(group["kwh"]*group['rates']*0.01))

    meter_energy["Meter"] =  m_lt['meter_id']
    meter_energy["Total_Consumption"] = kwh_sum
    meter_energy["Cost"] = cost_sum

    return meter_energy 


        
   
    

In [354]:
# Solution 1
meter_energy = calculate_cost_energy(m_lt, ft)
print(meter_energy)

/var/folders/yd/7fn380h1507crytk9ytqnc440000gn/T/ipykernel_68823/3518862690.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['rates'].loc[indices[i]] = rate['rate_p_per_kwh'].iloc[ac_index[-1]]


      Meter  Total_Consumption         Cost
0  14676236            28978.0   100.152012
1  34509937            78324.0   275.489119
2  50264822           265667.0   731.244071
3  88357331           484399.0  1433.160171


In [265]:
zones = np.unique(rate['exit_zone'])
print('\n ', zones)
print('\n ', len(zones))



  ['EA1' 'EA2' 'EA3' 'EA4' 'EM1' 'EM2' 'EM3' 'EM4' 'LC' 'LO' 'LS' 'LT' 'LW'
 'NE1' 'NE2' 'NE3' 'NO1' 'NO2' 'NT1' 'NT2' 'NT3' 'NW1' 'NW2' 'SC1' 'SC2'
 'SC4' 'SE1' 'SE2' 'SO1' 'SO2' 'SW1' 'SW2' 'SW3' 'WA1' 'WA2' 'WM1' 'WM2'
 'WM3']

  38


In [267]:
def random_meter_list(input_meters, total_zones):
    new_meter_list = pd.DataFrame()
    total_meter_cap = np.random.randint(20000, high=1000000,
                               size=input_meters)
    zone_indices = np.random.randint(0, high=total_zones,
                               size=input_meters)
    meter_names = np.random.randint(20000000,
                               size=input_meters)
    
    new_meter_list['meter_id'] = meter_names
    new_meter_list['aq_kwh'] = total_meter_cap
    new_meter_list['exit_zone'] = zones[zone_indices]
    return new_meter_list
    

In [304]:
# Solution 2
total_zones = len(zones)
input_meters = 2
new_meter_list = random_meter_list(input_meters, total_zones)
print('\n',new_meter_list)






    meter_id  aq_kwh exit_zone
0   2471705  358517       EA2
1   2865611  701648       WM3


In [309]:
def random_forcast_table(meter_ids, start_date, end_date):
    
    new_forcast_table = pd.DataFrame()
    dates = pd.date_range(start=start_date, end=end_date)
    all_meter = np.array([])
    all_date = []
    all_kwh = np.array([])
    for i in range(len(meter_ids)):
        all_meter = np.append(all_meter, np.ones(len(dates))*meter_ids[i])
        all_date.extend(dates)
        all_kwh = np.append(all_kwh, np.random.normal(50, 3, len(dates)))
        
    new_forcast_table['meter_id'] = all_meter.astype(np.int32)
    new_forcast_table['date'] = all_date
    new_forcast_table['kwh'] = all_kwh
    return new_forcast_table 

In [310]:
# Solution 3
meter_ids = new_meter_list['meter_id']
start_date = '1/1/2020'
end_date = '1/24/2020'
# start date and end date of all meters are same
print('\n', dates)
new_forcast_table = random_forcast_table(meter_ids, start_date, end_date)
print('\n', new_forcast_table)


 DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12',
               '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16',
               '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20',
               '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24'],
              dtype='datetime64[ns]', freq='D')

     meter_id       date        kwh
0    2471705 2020-01-01  45.705475
1    2471705 2020-01-02  49.403064
2    2471705 2020-01-03  46.183982
3    2471705 2020-01-04  49.952116
4    2471705 2020-01-05  53.781508
5    2471705 2020-01-06  54.130648
6    2471705 2020-01-07  52.259357
7    2471705 2020-01-08  48.236786
8    2471705 2020-01-09  50.036010
9    2471705 2020-01-10  53.442905
10   2471705 2020-01-11  53.234675
11   2471705 2020-01-12  52.830881
12   2471705 2020-01-13  49.610549
13   2471705 2020-01-14  53.843

In [353]:
# 2
total_zones = len(zones)
input_meters = 2
new_meter_list = random_meter_list(input_meters, total_zones)
print(new_meter_list,'\n')

# 3
meter_ids = new_meter_list['meter_id']
start_date = '5/5/2020'
end_date = '10/24/2024'

try:
    new_forcast_table = random_forcast_table(meter_ids, start_date, end_date)
    # print(new_forcast_table)
    
    # 4
    new_meter_energy = calculate_cost_energy(new_meter_list, new_forcast_table)
    print(new_meter_energy)
except:
    if (pd.to_datetime(start_date) < rate['date'].iloc[0]) | (pd.to_datetime(end_date) > rate['date'].iloc[-1]):
        print("Change the input the dates for calculating the transportation costs")
        
# start date and end date of all meters are same
# end date should be atleast equal latest date in rate_table to calculate rate
# start date should be more than the minimum in the rate table






   meter_id  aq_kwh exit_zone
0   3672824  842935       NT3
1   6036459  896415       SC2 



/var/folders/yd/7fn380h1507crytk9ytqnc440000gn/T/ipykernel_68823/3518862690.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['rates'].loc[indices[i]] = rate['rate_p_per_kwh'].iloc[ac_index[-1]]


     Meter  Total_Consumption        Cost
0  3672824       81750.618777  344.303558
1  6036459       81792.861395  343.129235


2020-05-05 00:00:00
